In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
            .master("local[*]") \
            .appName("test") \
            .getOrCreate()

25/03/08 11:39:05 WARN Utils: Your hostname, SRCIND-21BQ9G3 resolves to a loopback address: 127.0.1.1; using 172.26.144.30 instead (on interface eth0)
25/03/08 11:39:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/08 11:39:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet("data/pq/green/*/*")

## to implement

```sql
SELECT 
    date_trunc('hour', lpep_pickup_datetime) as hour, 
    PULocationID as zone,

    sum(total_amount) as revenue_monthly_total_amount,
    COUNT(1) as number_of_records
    FROM 
        green
    WHERE
        lpep_pickup_datetime > '2020-01-01 00:00:00'
    GROUP BY 
        1,2
```

In [6]:
rdd = df_green \
        .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
        .rdd

In [8]:
from datetime import datetime

In [12]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime > start

In [14]:
rows = rdd.take(5)
row = rows[0]
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)

datetime.datetime(2020, 1, 23, 13, 0)

In [22]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)

    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [25]:
def reduce(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value

    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [32]:
from collections import namedtuple

In [33]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [34]:
def unwrap(row):
    return RevenueRow(
        hour = row[0][0], 
        zone = row[0][1], 
        revenue = row[1][0], 
        count = row[1][1]
    )

In [38]:
from pyspark.sql import types

In [41]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True), 
    types.StructField('zone', types.IntegerType(), True), 
    types.StructField('revenue', types.DoubleType(), True), 
    types.StructField('count', types.IntegerType(), True)
])

In [42]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(reduce) \
    .map(unwrap) \
    .toDF(result_schema)

In [43]:
df_result.schema

StructType([StructField('hour', TimestampType(), True), StructField('zone', IntegerType(), True), StructField('revenue', DoubleType(), True), StructField('count', IntegerType(), True)])

In [46]:
df_result.write.parquet("tmp/green-revenue")

25/03/08 12:30:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [51]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [58]:
def apply_mode_in_batch(partition):
    count = 0

    for _ in partition:
        count = count + 1
    return [count]

In [59]:
duration_rdd.mapPartitions(apply_mode_in_batch).collect()

[746744, 418184, 219219, 215981, 232942, 214666, 178149, 78632]

In [60]:
import pandas as pd

In [61]:
rows = duration_rdd.take(100)

In [65]:
# pd.DataFrame(rows, columns = columns)

In [63]:
def apply_mode_in_batch(partition):
    df = pd.DataFrame(partition, columns = columns)
    count = len(df)
    return [count]

In [64]:
duration_rdd.mapPartitions(apply_mode_in_batch).collect()

[746744, 418184, 219219, 215981, 232942, 214666, 178149, 78632]

In [66]:
def model_predict(df):
    y_pred = df.trip_distance * 5
    return y_pred


In [72]:
def apply_mode_in_batch(partition):
    df = pd.DataFrame(partition, columns = columns)
    df['lpep_pickup_datetime'] = df['lpep_pickup_datetime'].astype(str) # Spark does not recognise the Pandas Timestamp, converting it to string will solve the empty issues
    predictons = model_predict(df)
    df['predicted_duration'] =  predictons

    for row in df.itertuples():
        yield row

In [73]:
df_predicts = duration_rdd \
                .mapPartitions(apply_mode_in_batch) \
                .toDF() \
                .drop('Index')

In [74]:
df_predicts.show()

[Stage 31:>                                                         (0 + 1) / 1]

+--------+--------------------+------------+------------+-------------+------------------+
|VendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|predicted_duration|
+--------+--------------------+------------+------------+-------------+------------------+
|     2.0| 2020-01-23 13:10:15|          74|         130|        12.77|63.849999999999994|
|     NaN| 2020-01-20 15:09:00|          67|          39|          8.0|              40.0|
|     2.0| 2020-01-15 20:23:41|         260|         157|         1.27|              6.35|
|     2.0| 2020-01-05 16:32:26|          82|          83|         1.25|              6.25|
|     2.0| 2020-01-29 19:22:42|         166|          42|         1.84| 9.200000000000001|
|     2.0| 2020-01-15 11:07:42|         179|         223|         0.76|               3.8|
|     2.0| 2020-01-16 08:22:29|          41|         237|         3.32|16.599999999999998|
|     2.0| 2020-01-28 17:05:28|          75|         161|         2.21|             11.05|